In [5]:
pip install python-telegram-bot openpyxl pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install PyTelegramBotAPI openpyxl pandas

     ---------------------------------------- 0.0/48.3 kB ? eta -:--:--
     ---------------------------------------- 48.3/48.3 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/287.4 kB ? eta -:--:--
   ---------------------------------------  286.7/287.4 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 287.4/287.4 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/105.4 kB ? eta -:--:--
   ---------------------------------------- 105.4/105.4 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/128.7 kB ? eta -:--:--
   ---------------------------------------- 128.7/128.7 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes, MessageHandler, filters, ConversationHandler
import pandas as pd
import os

# Загружаем Excel
df = pd.read_excel('products.xlsx')

# Получаем уникальные категории
categories = df['Категория'].unique()

# Этапы разговора
REVIEW = 1  # Этап, когда пользователь пишет отзыв

# Команда /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = [
        [InlineKeyboardButton("Купить", callback_data='buy')],
        [InlineKeyboardButton("Оставить отзыв", callback_data='review')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text("Добро пожаловать в наш магазин! Что вы хотите сделать?", reply_markup=reply_markup)

# Обработка кнопок
async def button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    data = query.data

    # Покупка — показать категории
    if data == 'buy':
        keyboard = [[InlineKeyboardButton(cat, callback_data=f"cat_{cat}_0")] for cat in categories]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await query.edit_message_text("Выберите категорию:", reply_markup=reply_markup)

    # Назад к категориям
    elif data == 'back_to_categories':
        keyboard = [[InlineKeyboardButton(cat, callback_data=f"cat_{cat}_0")] for cat in categories]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await query.edit_message_text("Выберите категорию:", reply_markup=reply_markup)

    # Просмотр товаров в категории с пагинацией
    elif data.startswith('cat_'):
        parts = data.split('_')
        category = parts[1]
        page = int(parts[2])

        products = df[df['Категория'] == category]
        products_per_page = 5
        start_idx = page * products_per_page
        end_idx = start_idx + products_per_page

        page_products = products.iloc[start_idx:end_idx]
        text = f"Товары в категории: {category}\n\n"
        keyboard = []

        for _, row in page_products.iterrows():
            text += f"{row['Название детали']} (Артикул: {row['Артикул']})\nЦена: {row['Цена']} руб.\n\n"
            keyboard.append([InlineKeyboardButton(row['Название детали'], callback_data=f"prod_{row['Артикул']}")])

        nav_buttons = []
        if page > 0:
            nav_buttons.append(InlineKeyboardButton("Назад", callback_data=f"cat_{category}_{page-1}"))
        else:
            nav_buttons.append(InlineKeyboardButton("Назад к категориям", callback_data="back_to_categories"))
        if end_idx < len(products):
            nav_buttons.append(InlineKeyboardButton("Далее", callback_data=f"cat_{category}_{page+1}"))

        if nav_buttons:
            keyboard.append(nav_buttons)

        reply_markup = InlineKeyboardMarkup(keyboard)
        await query.edit_message_text("Выберите товар или используйте навигацию:", reply_markup=reply_markup)

    # Просмотр товара
    elif data.startswith('prod_'):
        product_id = int(data.split('_')[1])
        product = df[df['Артикул'] == product_id].iloc[0]
        text = f"{product['Название детали']}\nАртикул: {product['Артикул']}\nЦена: {product['Цена']} руб."
        photo_path = os.path.join('photos', product['Фото'])

        if os.path.exists(photo_path):
            with open(photo_path, 'rb') as photo:
                await query.message.reply_photo(photo=photo, caption=text)
        else:
            await query.message.reply_text(text)

    # Оставить отзыв
    elif data == 'review':
        await query.edit_message_text("Напишите ваш отзыв, и мы с радостью ответим!")

        # Начинаем процесс отзыва
        return REVIEW

# Обработка текста отзыва
async def handle_review(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_review = update.message.text
    if user_review.strip():  # Проверка, что отзыв не пустой
        await update.message.reply_text("Вау, ну ты красава, братан!")
    else:
        await update.message.reply_text("Не хочешь оставить отзыв?")

    # Возвращаемся в начальный статус
    return ConversationHandler.END

# Завершение разговора
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Отзыв не оставлен. Чем могу помочь еще?")
    return ConversationHandler.END

# Запуск бот
def main():
    TOKEN = "ВАШ_ТОКЕН_ЗДЕСЬ"  # Замените на свой токен
    application = Application.builder().token(TOKEN).build()

    # Создание ConversationHandler для обработки отзыва
    conversation_handler = ConversationHandler(
        entry_points=[CallbackQueryHandler(button, pattern='^review$')],
        states={
            REVIEW: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_review)],
        },
        fallbacks=[CommandHandler("cancel", cancel)],
    )

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CallbackQueryHandler(button))
    application.add_handler(conversation_handler)

    application.run_polling()
    if name == '__main__':
        main()


In [ ]:
from telegram.ext import Application

TOKEN = "7713900787:AAHuEPpjV4Kgk5hPEA_dMTdQFOo9Z2yL7CM"

application = Application.builder().token(TOKEN).build()
application.add_handler(CommandHandler("start", start))
application.add_handler(CallbackQueryHandler(button))

await application.initialize()
await application.start()
await application.updater.start_polling()


<Queue at 0x26f41112450 maxsize=0 _getters[1]>

Exception happened while polling for updates.
Traceback (most recent call last):
  File "c:\Users\horol\AppData\Local\Programs\Python\Python311\Lib\site-packages\telegram\ext\_utils\networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "c:\Users\horol\AppData\Local\Programs\Python\Python311\Lib\site-packages\telegram\ext\_utils\networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\horol\AppData\Local\Programs\Python\Python311\Lib\site-packages\telegram\ext\_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\horol\AppData\Local\Programs\Python\Python311\Lib\site-packages\telegram\ext\_extbot.py", line 669, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\horol\AppData\Local\Programs\Python\Python311